# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd

In [ ]:
# Importing the dataset from Ironhack's database
pokemon = pd.read_csv('Pokemon.csv') 
pokemon

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [ ]:
pokemon = pokemon.drop(columns="Hypothesis")
pokemon

In [ ]:
# your code here
pokemon['Hypothesis']= pokemon.iloc[:, -7:-1].sum(axis=1)
pokemon

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [ ]:
# your code here
def name_combo(row):
    if pd.isnull(row['Type 2']):
        return row['Type 1']
    else:
        return row['Type 1'] + '-' + row['Type 2']
pokemon['NameCombo']=pokemon[['Type 1', 'Type 2']].apply(name_combo, axis = 1)

In [ ]:
pokemon['Type 1'].unique()

encoding_table = pokemon[['Type 1', 'Type 2']]

encoding_table


def name_combo2(row):
    if pd.isnull(row['Type 2']):
        return row['Type 1']
    else:
        return row['Type 1'] + '-' + row['Type 2']
encoding_table['NameCombo']=encoding_table[['Type 1', 'Type 2']].apply(name_combo2, axis = 1)
encoding_table

In [ ]:
encoding_table

# Get one hot encoding of columns B
one_hot = pd.get_dummies(encoding_table['Type 1'])
# Drop column B as it is now encoded
encoding_table = encoding_table.drop('Type 1',axis = 1)
# Join the encoded df
encoding_table = encoding_table.join(one_hot)
encoding_table

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict

d = defaultdict(LabelBinarizer)

cols2bnrz = ['Type 1','Type 2']

In [8]: df[cols2bnrz].apply(lambda x: d[x.name].fit(x))

    
In [10]: new = pd.DataFrame({
    ...:     'Reference': [6, 7],
    ...:     'Brand': ['Volvo', 'Audi'],
    ...:     'Town': ['Stockholm', 'Munich']
    ...: })

In [11]: new

In [12]: pd.DataFrame(d['Brand'].transform(new['Brand']), columns=d['Brand'].classes_)

In [13]: pd.DataFrame(d['Town'].transform(new['Town']), columns=d['Town'].classes_)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [ ]:
# Get all the unique values if we don't have them
unique_values = pd.unique(tmp_ohe1.values.ravel()) 
unique_values

In [ ]:
#encoding_table = encoding_table.replace(np.nan, '', regex=True)
tmp_ohe = pokemon[['Type 1', 'Type 2']]
tmp_ohe

In [ ]:

# Get all the unique values if we don't have them
unique_values = pd.unique(tmp_ohe1.values.ravel()) 

ohe = OneHotEncoder(categories=[unique_values]*len(tmp_ohe1), sparse=False)
encoded = pd.DataFrame(ohe.fit_transform(tmp_ohe1), columns=ohe.get_feature_names(tmp_ohe1.columns))

In [ ]:
encoding_table = pokemon[['Type 1', 'Type 2']]

encoding_table

In [ ]:
tmp_ohe
import numpy as np
tmp_ohe1 = tmp_ohe.replace(np.nan, 'a', regex=True)

In [ ]:
tmp_ohe1

In [86]:
p_t1 = pokemon[['Name','Type 1']]

In [85]:
p_t1





import pandas
from sklearn import preprocessing 

pd.get_dummies(p_t1,prefix=['Type 1'])

ValueError: Length of 'prefix' (1) did not match the length of the columns being encoded (2).

In [90]:
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 
  
#creating one hot encoder object by default 
# entire data passed is one hot encoded 
onehotencoder = OneHotEncoder() 
  
data = onehotencoder.fit_transform(p_t1).toarray() 


In [91]:
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
encoding_table

In [ ]:
pd.get_dummies(encoding_table,dummy_na=True)

In [ ]:
pokemon['Type 1'].unique()


'Grass'
'Poison'
'Fire' 
'Flying'
'Dragon'
'Water'
'Bug' 
'Normal'
'Electric'
'Ground' 
'Fairy' 
'Fighting' 
'Psychic' 
'Rock' 
    
    
    
    
    
    
    
    
    'Ghost', 'Ice',
       '', 'Dark', 'Steel', ''], dtype=object)



## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [ ]:
# your code here


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here